In [15]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [24]:
args = Namespace(
    raw_train_dataset_csv="data/yelp/raw_train.csv",
    raw_test_dataset_csv="data/yelp/raw_test.csv",
    proportion_subset_of_train=0.1,
    train_proportion=0.7,
    val_proportion=0.15,
    test_proportion=0.15,
    output_munged_csv="data/yelp/reviews_with_splits_lite.csv",
    seed=1337
)

In [37]:
# 원본 데이터를 읽습니다
train_reviews = pd.read_csv(args.raw_train_dataset_csv,header=None, names=['rating', 'review'], engine = 'python',encoding='utf-8',error_bad_lines=False)

# pd.read_cvs()에서 error_bad_lines = False 로 설정해서 오류가 있는 줄을 skip해서 데이터를 받았다. 이것때문에 구글링 엄청 했다.
# 추가로 header = None, names = [~] 를 하면 ,로 구분되는 csv 파일에 있어서 아무 열에도 이름이 없으니 해당 열에 해당 이름을 붙인다는 뜻이다.

Skipping line 515818: unexpected end of data


# # 읽어온 데이터를 by_rating에 딕셔너리로 저장한다


In [50]:
# 리뷰 클래스 비율이 동일하도록 만듭니다
by_rating = collections.defaultdict(list) # default값을 지정할 수 있는 dict -> list, set 등을 정리할때 유용하다
for _, row in train_reviews.iterrows(): # iterrows() -> index와 해당 행에 접근)(한 줄씩 접근)
    by_rating[row.rating].append(row.to_dict()) # 판다스에서 df.열이름 -> 해당 열에 접근할 수 있다, (==df['열이름'])
                                # df.to_dict() -> 사전형으로 변환, 이때 이중 딕셔너리 구조('열 1' : {인덱스 : 값 ...}, '열 2' : )
                                # 이때 해당 코드는 iterrows로 한 줄씩 가져와서 사전으로 바꾸므로 그냥 딕셔너리 구조를 가짐
# 딕셔너리 by_rating은 딕셔너리 구조를 원소로 갖는 리스트를 가지는 딕셔너리 key '1','2'를 가진다.
review_subset = []
for _, item_list in sorted(by_rating.items()): #by_rating.items() -> key와 value 쌍 -> key => 1,2 / value => 딕셔너리의 리스트
    # 즉, 딕셔너리의 리스트를 정렬한다는 뜻!
    
    n_total = len(item_list) # 리스트가 원소로 가지는 딕셔너리의 갯수
    n_subset = int(args.proportion_subset_of_train * n_total) # lite버전이므로 데이터의 10퍼센트만 사용한다는 뜻(0.1 * 갯수)
    review_subset.extend(item_list[:n_subset]) # extend 와 append의 차이점 => extend는 반복자의 원소를 모두 삽입, append는 반복자 하나를 원소 자체로 삽입
    
review_subset = pd.DataFrame(review_subset)

1
1


In [39]:
review_subset.head()

,rating,review
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,1,I don't know what Dr. Goldberg was like before...
2,1,I'm writing this review to give you a heads up...
3,1,Wing sauce is like water. Pretty much a lot of...
4,1,Owning a driving range inside the city limits ...


In [87]:
print(train_reviews.rating.value_counts()) # df.value_counts() -> 유일한 값의 원소의 갯수 / df.unique -> 유일한 값의 갯수(고유갯수)
print(review_subset.rating.value_counts())
# 전체 갯수와 우리가 사용할 데이터 갯수 비교

1    259235
2    256582
Name: rating, dtype: int64
1    25923
2    25658
Name: rating, dtype: int64


판다스에서 딕셔너리로 만들어진 시리즈는 데이터프레임의 하나의 열이라고 할 수 있다.  

즉, {'rating' : [1,1,1,2,2,2,2.....]}, {'review' : ["\~","\~"]} 이 두 시리즈가 열이 되어 합쳐져 하나의 데이터 프레임을 만드는 것이다.  

{'a' : 1, 'b' : 2 ....}처럼 하나의 딕셔너리에 여러 key가 있다면 이 시리즈는 key가 index가 되고 value가 값이 되는 데이터 프레임을 만든다.

In [116]:
# 훈련, 검증, 테스트를 만들기 위해 별점을 기준으로 나눕니다
by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

# 분할 데이터를 만듭니다.
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):

    np.random.shuffle(item_list)
    
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)
    
    # 데이터 포인터에 분할 속성을 추가합니다
    for item in item_list[:n_train]: # item_list 는 딕셔너리를 원소로 가지는 리스트 따라서 item 은 딕셔너리이다.
                                    ### ([{랭크 : ~, 리뷰 : ~},{랭크 : ~, 리뷰 : ~},{랭크 : ~, 리뷰 : ~},,,])###
        item['split'] = 'train'
    
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
        
    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    # 최종 리스트에 추가합니다
    final_list.extend(item_list)

In [117]:
final_reviews = pd.DataFrame(final_list)

In [118]:
final_reviews.split.value_counts()

train    36106
val       7736
test      7736
Name: split, dtype: int64

In [119]:
final_reviews.rating.value_counts()

1    25923
2    25658
Name: rating, dtype: int64

In [120]:
# 리뷰를 전처리합니다
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text
    
final_reviews.review = final_reviews.review.apply(preprocess_text)

In [112]:
final_reviews['rating'] = final_reviews.rating.apply({1: 'negative', 2: 'positive'}.get)
# get -> 어떤 키 값에 대해 value를 반환한다. 이때 데이터프레임의 rating에 대해서 {}.get(rating) 이 적용된다고 생각하자.
#(apply는 기본적으로 모든 행을 기준으로 내부 함수를 전역에 적용한다. 즉, get의 파라미터로 rating(1 or 2)가 들어간다.)
# 따라서 해당 코드는 rating을 하나씩 뒤져서 {}.get()에 파라미터로 넣고, rating이 1 이면 그 value인 negative가 반환되서 해당 열에
# = 를 통해 할당되는 방식으로 작동한다.

In [124]:
final_reviews = final_reviews.replace({'rating' : {1 : 'negative', 2 : 'positive'}}) # 내가 만든 데이터 치환 코드

In [125]:
final_reviews.head()

,rating,review,split
0,negative,worst climbing gym i have ever been to . custo...,train
1,negative,if i could do zero stars i would . i dropped m...,train
2,negative,to sum this whole paragragh up . . . . . . n b...,train
3,negative,my first review of china chili was stars . aft...,train
4,negative,a tourist trap that will nickel and dime you t...,train


In [126]:
final_reviews.rating

0        negative
1        negative
2        negative
3        negative
4        negative
           ...   
51576    positive
51577    positive
51578    positive
51579    positive
51580    positive
Name: rating, Length: 51581, dtype: object

In [127]:
final_reviews['rating']

0        negative
1        negative
2        negative
3        negative
4        negative
           ...   
51576    positive
51577    positive
51578    positive
51579    positive
51580    positive
Name: rating, Length: 51581, dtype: object

In [128]:
final_reviews.to_csv(args.output_munged_csv, index=False)